In [1]:
import pandas as pd
import numpy as np
import altair as alt

from collections import OrderedDict

In [4]:
sample_data = pd.read_csv('./test_data/women_comp_india_static_plot_data.csv')
sample_data = sample_data.drop(['Unnamed: 0'], axis=1)


sample_data.head()

,num_paragraph,section,section_paragraph,occurence
0,0,abstract,0,2
1,1,abstract,1,0
2,2,introduction,0,0
3,3,introduction,1,0
4,4,introduction,2,0


In [87]:
section_order = sample_data.sort_values('num_paragraph').section.unique().tolist()
section_order_num = []

In [114]:
# temporarily removed 'scale=alt.Scale(domainMin= 0)' for alt,Y, b/c it fails to run on my conda env; I will figure out a workaround later
def create_bar_chart(sample_data):
    max_occurence = sample_data.occurence.max()
    chart = alt.Chart(sample_data).mark_bar(
             width=6
    ).encode(
        x=alt.X('section:O', title='Paper Section',
                axis=alt.Axis(grid=False),sort=list(OrderedDict.fromkeys(sample_data['section']))),
        y=alt.Y('sum_occ:Q', title='Key Word Occurence',
                axis=alt.Axis(grid=False),
               scale=alt.Scale(domainMin=0)),
        color='section',
        tooltip=['section:N', 'sum_occ:Q']
    ).transform_aggregate(
    sum_occ='sum(occurence)',
    groupby=["section"]
).properties(
        width=200,
        height=250
    )

    return chart

In [115]:
def create_staggered_bead(sample_data):
    max_paragraph = sample_data.num_paragraph.max()
    base = alt.Chart(sample_data).mark_circle(opacity=1 ).encode(    #stroke='#4c78a8'
        x=alt.X('num_paragraph:O', title='Paragraph Number', 
                axis=alt.Axis(grid=False,
                              values=list(range(1, max_paragraph+1, 5)), labelAngle=0)),
        y=alt.Y('section:N', sort=list(OrderedDict.fromkeys(sample_data['section']))), 
        size=alt.Size('occurence:Q', title="Key Word Occurence"),
        color = 'section',
        tooltip=['section:N','section_paragraph:Q', 'occurence:Q']
    ).properties(
        width=500,
        height=250
    )
    lines = alt.Chart(sample_data).mark_tick(orient='horizontal').encode( #.mark_tick(stroke='#4c78a8')
        x=alt.X('num_paragraph:O', title='Paragraph Number', 
                axis=alt.Axis(grid=False,
                              values=list(range(1, max_paragraph+1, 5)), labelAngle=0)), 
        y=alt.Y('section:N',title='Paper Section', sort=list(OrderedDict.fromkeys(sample_data['section']))),
        color = 'section'
    ).properties(
        width=500,
        height=250
    )
    return base + lines

In [116]:
create_staggered_bead(sample_data) | create_bar_chart(sample_data)

alt.HConcatChart(...)

### Some Ideas

- What if we added a click selection that would show all the citations within the paragraph selected as a related work subpanel?
- If we can parse the figures out, maybe we can add related figure information to the hover interaction? So you hover over a paragraph and it tells you the section, text paragraph number, as well as the related figure number
- For the default, do we want to suggest the top N number of keywords while displaying the plots for the top word? But then allow the user to click or input some of the other ones